In [ ]:
%matplotlib inline
import librosa, librosa.display
import IPython.display as ipd
import vamp
import matplotlib.pyplot as plt
import numpy as np
import scipy.fftpack as fft

In [ ]:
import re
import collections

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pprint = pp.pprint

In [ ]:
import vamp

In [ ]:
#path = '/home/sonja/Dropbox/RC/Passive ear trainer/Songs/Autumn Leaves - Yenne Lee plays 2004 Pepe Romero Jr.-HxGT5z6d-GA.m4a'
#path = '/home/sonja/Dropbox/RC/Passive ear trainer/Songs/Beatles - I wanna hold your hand - live.m4a'
#path = '/home/sonja/Dropbox/RC/Passive ear trainer/Songs/Beatles - I wanna hold your hand -studio.m4a'
#path = '/home/sonja/Dropbox/RC/Passive ear trainer/Songs/Dumb Ways to Die-IJNR2EpS0jw.m4a'
#path = '/home/sonja/Dropbox/RC/Passive ear trainer/Songs/Queen - I Want to Break Free (Official Lyric Video)-WUOtCLOXgm8.m4a'
#path = '/home/sonja/Dropbox/RC/Passive ear trainer/Songs/Selena Gomez, Marshmello - Wolves (Official Music Video)-cH4E_t3m3xM.m4a'
path = '/home/sonja/Dropbox/RC/Passive ear trainer/Songs/Ska-p Los hijos bastardos de la globalizacion con Letra-upnPasIYeMc.m4a'
#path = '/home/sonja/Dropbox/RC/Passive ear trainer/Songs/The Killers - Human-RIZdjT1472Y.m4a'
#path = '/home/sonja/Dropbox/RC/Passive ear trainer/Songs/THE MUFFIN SONG (asdfmovie feat. Schmoyoho)-LACbVhgtx9I.m4a'

In [ ]:
#HAS TO BE RUN FOR EVERY SONG
song = librosa.load(path)

In [ ]:
cow = librosa.load('/home/sonja/Dropbox/RC/Passive ear trainer/Songs/Kuh muht-TdheW61w4Co.m4a')

In [ ]:
song_analysis = vamp.collect(song[0], song[1], "nnls-chroma:chordino")['list']
pprint(list(enumerate(song_analysis)))

In [ ]:
def extend_short_rec(short_rec, desired_num_samples, desired_vol):
    bounds = [1113687, 1148687] #FIXME
    current_vol = max(short_rec[0])
    
    sample = last_not_0_before_par(short_rec, bounds[1])
    for _ in range(50):
        sample = sign_change(short_rec, sample-1)
    end_copy = sample

    for _ in range(1000):
        sample = sign_change(short_rec, sample-1)
    beginning_copy = sample

    extension = short_rec[0][bounds[0]:end_copy]
    while len(extension) < desired_num_samples:
        extension = np.concatenate((extension, short_rec[0][beginning_copy:end_copy]))
        
    extension = extension/current_vol*desired_vol
    
    return extension[:desired_num_samples]

In [ ]:
def cow_generator(amp, rate):
    if rate != cow[1]:
        return False
    def f(num_samples):
        return extend_short_rec(cow, num_samples, amp)
    return f 

In [ ]:
def last_not_0_before_par(rec, par):
    ind = par
    while rec[0][ind] == 0:
        ind -= 1
    return ind

def first_0_before_par(rec, par):
    ind = par
    while rec[0][ind] != 0:
        ind -= 1
    return ind

def sign_change(rec, par):
    ind = par
    while rec[0][ind] * rec[0][par] > 0:
        ind -= 1
    return ind

In [ ]:
def note_by_num_samples(freq, num_samples, amp=1, rate=song[1]):
    data = np.array([np.sin(2*np.pi*freq*librosa.samples_to_time(sample, rate))*amp for sample in range(num_samples)], dtype=np.float32)
    return data

In [ ]:
def synth_generator(freq, amp, rate):
    def f(num_samples):
        return note_by_num_samples(freq, num_samples, amp, rate)
    return f

In [ ]:
encr_notes = {'C':0, 'C#':1, 'Db':1, 'D':2, 'D#':3, 'Eb':3, 'E':4, 'Fb':4, 'E#':5, 'F':5, 'F#':6, 'Gb':6, 'G':7, 'G#':8, 'Ab':8, 'A':9, 'A#':10, 'Bb':10, 'B':11, 'Cb':11, 'B#':0}

depr_flat_notes = ['C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B']
decr_sharp_notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

In [ ]:
scale_deg_in_semitones = {1:0, 2:2, 3:4, 4:5, 5:7, 6:9, 7:11}

In [ ]:
#given a tonality and a scale degree, find the chord corresponding to the scale degree in the given tonality
def absolute_chord(tonality, scale_deg):
    encr_chord = (encr_notes[tonality] + scale_deg_in_semitones[scale_deg]) % 12
    if len(tonality) > 1 and tonality[1] == 'b':
        chord = decr_flat_notes[encr_chord]
    else:
        chord = decr_sharp_notes[encr_chord]
    return chord

In [ ]:
bass_freq = {'C':65.41, 'C#':69.30, 'D':73.42, 'D#':77.78, 'E':82.41, 'F':87.31, 'F#':92.50, 'G':98.00, 'G#':103.83, 'A':110.00, 'A#':116.54, 'B':123.47}
high_pitch_freq = {'C':130.81, 'C#':138.59, 'D':146.83, 'D#':155.56, 'E':164.81, 'F':174.61, 'F#':185.00, 'G':196.00, 'G#':207.65, 'A':220.00, 'A#':233.08, 'B':246.94}

In [ ]:
def find_bounds_samples(song_analysis, chord):
    #bounds_chord = [librosa.time_to_samples(t, song[1]) for t in timestamps_chord]

    pairs = list(zip(song_analysis[:-1], song_analysis[1:]))
    time_bounds = [(float(item1['timestamp']), float(item2['timestamp'])) for item1, item2 in pairs if re.match(re.escape(chord)+r'.*', item1['label'])]
    bounds = [librosa.time_to_samples(t, song[1])for t in time_bounds]
    return bounds

In [ ]:
def find_amplitude_rel_song(song, strength):
    song_intensity = max(song[0])
    return song_intensity*strength

In [ ]:
def generate_audio_to_add(song_analysis, song_length, chord, timber, amp, rate):
    if timber == 'cow':
        note_generator = cow_generator(amp, rate)
        if not note_generator:
            'Sorry, the cow is tired of mooing. Please, choose a different timber!'
    else:
        if timber == 'low_pitch_synth':
            freq = bass_freq[chord]
        else:
            freq = high_pitch_freq[chord]
        note_generator = synth_generator(freq, amp, rate)
    
    bounds_chord = find_bounds_samples(song_analysis, chord)
    add_to_song = np.zeros(song_length)
    for beginning, end in bounds_chord:
        if end > song_length:
            end = song_length
        add = note_generator(end - beginning)
        add_to_song[beginning:end] = add
        
    return add_to_song

In [ ]:
def find_tonality(song_analysis):
    #look for a dominant chord: TO DO
    
    #find last chord
    last_chord_index = -1
    while song_analysis[last_chord_index]['label'] == 'N':
        last_chord_index -= 1
    last_chord = song_analysis[last_chord_index]['label']
    
    # find the chord that is played the most
    list_of_chords = [chord['label'] for chord in song_analysis]
    numb_appearance_chord2 = collections.Counter(list_of_chords)
    most_appearing_chord = numb_appearance_chord2.most_common(1)[0][0]


    sign = ''
    if len(most_appearing_chord) > 1:
        if most_appearing_chord[1] == '#' or most_appearing_chord[1] == 'b':
            sign = most_appearing_chord[1]

    tonality = most_appearing_chord[0] + sign
    
    certainty = True
    if last_chord != most_appearing_chord:
        certeinty = False
        
    return tonality, certainty

In [ ]:
scale_deg = 1 # 1,2,3,4,5,6 or 7
timber = 'low_pitch_synth' # 'low_pitch_synth', 'high_pitch_synth' or 'cow'
amp = 1/2 # 1/2, 1, 2 

tonality, certainty = find_tonality(song_analysis)
if not certainty:
    print('The tonality is probably wrong, sorry for that!')

chord = absolute_chord(tonality, scale_deg)
length = len(song[0])
to_add = generate_audio_to_add(song_analysis, length, chord, timber, amp, song[1])



In [ ]:
output = song[0] + to_add
print(len(to_add))
print(len(song[0]))
print(tonality)
bounds = find_bounds_samples(song_analysis, tonality)
beginning, end = bounds[4]
ipd.Audio(output, rate=song[1])

The rest is not necessary anymore

In [ ]:
#HAS TO BE RUN FOR EVERY SONG
#Compute the chord that corresponds to a certain scale degree:
scale_degree = 1
chord = notes_sharp_medium_pitch[(encr_notes[tonality] + scale_degree_in_semitones[scale_degree]) % 12]
print(chord)

In [ ]:
#HAS TO BE RUN FOR EVERY SONG
#Find the time stamps in between which the chord corresponding to the chosen scale degree appears in the song:
chord_pairs = list(zip(chords[:-1], chords[1:]))
timestamps_chord = [ (float(c[0]['timestamp']), float(c[1]['timestamp'])) for c in chord_pairs if re.match(re.escape(chord[0])+r'.*', c[0]['label'])]
bounds_chord = [librosa.time_to_samples(t, song[1]) for t in timestamps_chord]
lables_chord = [c[0]['label'] for c in chord_pairs if re.match(re.escape(chord[0])+r'.*', c[0]['label'])]

#chord_number = 2
#print(lables_chord[chord_number])
#ipd.Audio(song[0][bounds_chord[chord_number][0]:bounds_chord[chord_number][1]], rate=song[1])


In [ ]:
#Has TO BE TUN FOR EVERY SONG
#Compute highest amplitude in the song:
max_amplitude = max(song[0])
print(max_amplitude)

In [ ]:
#HAS TO BE RUN FOR EVERY SONG
#Generate the output (audio of song plus the base note of the chord corresponding to the scale degree chosen every time that chord appears):
add_to_song = np.zeros(len(song[0]))
for beginning, end in bounds_chord:
    if end > len(song[0]):
        end = len(song[0])
    add = note_by_num_samples(chord[1] , end-beginning, amp=(max_amplitude)*5)
    add_to_song[beginning:end] = add
    
output = song[0] + add_to_song
ipd.Audio(output, rate=song[1])

In [ ]:
add_cow = np.zeros(len(song[0]))
max_num_samples = max([end-beginning for beginning, end in bounds_chord])
long_cow = long_enough_ignoring_frequencies(max_num_samples)
for beginning, end in bounds_chord:
    if end > len(song[0]):
        end = len(song[0])
    add = long_cow[:(end-beginning)]
    add_cow[beginning:end] = add

output = song[0] + add_cow
ipd.Audio(output, rate=song[1])

In [ ]:
#Generate the output (audio of song plus the base note of the chord corresponding to the scale degree chosen every time that chord appears):
add_to_song = np.zeros(len(song[0]))
for beginning, end in bounds_chord:
    if end > len(song[0]):
        end = len(song[0])
    add = note_by_num_samples(chord[1] , end-beginning, amp=max_amplitude)
    add_to_song[beginning:end] = add
    
output = song[0] + add_to_song
ipd.Audio(output, rate=song[1])

In [ ]:
#computations for for getting the bounds of the cow sample between which the cow moos
cow_chords = vamp.collect(cow[0], cow[1], "nnls-chroma:chordino")['list']
label = 'N'
ind = 0
while label == 'N':
    ind += 1
    label = cow_chords[ind]['label']
beginning_ind = ind
print(beginning_ind)
while label != 'N':
    ind += 1
    label = cow_chords[ind]['label']
end_ind = ind - 1
print(end_ind)

timestamps_cow = [float(cow_chords[beginning_ind]['timestamp']), float(cow_chords[2]['timestamp'])]
bounds_cow = [librosa.time_to_samples(t, cow[1]) for t in timestamps_cow]

bounds_cow[0]=bounds_cow[0]-25000
bounds_cow[1]=bounds_cow[0]+35000
print(bounds_cow)
cow_in_F = cow[0][bounds_cow[0]:bounds_cow[1]]
ipd.Audio(cow_in_F, rate=cow[1])

In [ ]:
#for cow
last_int_sample = last_not_0_before_par(bounds_cow[1])
sample = last_int_sample
for _  in range(50):
    sample = sign_change(sample-1)
last_sample_extension = sample

for _  in range(1000):
    sample = sign_change(sample-1)
first_sample_extension = sample
    
#first_sample_extension = sample
print(last_int_sample)
print(first_sample_extension)

long_sample_cow = cow[0][bounds_cow[0]:bounds_cow[1]]
for _ in range(10):
    long_sample_cow = np.concatenate((long_sample_cow, cow[0][first_sample_extension:last_sample_extension]))
print(len(long_sample_cow))
ipd.Audio(long_sample_cow, rate=cow[1])

In [ ]:
#for cow
def long_enough_ignoring_frequencies(num_samples):
    output = cow[0][bounds_cow[0]:bounds_cow[1]]
    while len(output) < num_samples:
        output = np.concatenate((output, cow[0][first_sample_extension:last_sample_extension]))
    return output